# Session 4: Visualization of outputs
You will learn how to visualize the input/output datasets



In [ ]:
# import pyIncore modules
from pyincore import IncoreClient, FragilityService, FragilityCurveSet, MappingSet, Dataset

In [ ]:
# Connect to IN-CORE service
client = IncoreClient()

In [ ]:
fragility_services = FragilityService(client)

## 1. Joining Dataset

Data preparation and data post-processing are common procedures. Prior to using pyIncore, users often encounter situation that they need to reshape their own dataset to make it compliant with the input dataset format of pyIncore. 

For example, adding GUID to inventory dataset. GUID (aka UUID) is an acronym for 'Globally Unique Identifier'. It is a 128-bit integer number used to identify resources. In IN-CORE We **require** each individual row/item in an inventory to have its own GUID. If your dataset does not come with GUID, pyIncore has utility method to help you add it.

After acquiring outputs from pyIncore analyses, often time user would need to perform data aggregation to gain statitical insights. The below tutorial gives a few examples on how to join datasets and generate some basic visualizations.

### Step 1: Running building dmg analysis (Tornado)

In [ ]:
# create a local dataset with buiding inventory data
local_buildings = Dataset.from_file("sample_bldgs_w_guid.shp", data_type="ergo:buildingInventoryVer7")

In [ ]:
# import building damage analysis class
from pyincore.analyses.buildingdamage import BuildingDamage

In [ ]:
# Define the variables to be used for input parameters
hazard_type = "earthquake"
hazard_id = "5b902cb273c3371e1236b36b"
result_name = "memphis_eq_bldg_dmg_result"
mapping_id = "5b47b350337d4a3629076f2c"

# create fragility mapping object with id
eq_mapping_set = MappingSet(fragility_services.get_mapping(mapping_id))

# Create building damage analysis object
eq_bldg_dmg = BuildingDamage(client)

eq_bldg_dmg.set_input_dataset("buildings", local_buildings)
eq_bldg_dmg.set_input_dataset("dfr3_mapping_set", eq_mapping_set)
eq_bldg_dmg.set_parameter("result_name", result_name)
eq_bldg_dmg.set_parameter("hazard_type", hazard_type)
eq_bldg_dmg.set_parameter("hazard_id", hazard_id)
eq_bldg_dmg.set_parameter("num_cpu", 4)

# Run building damage analysis
eq_bldg_dmg.run_analysis()

In [ ]:
eq_bldg_dmg.get_output_dataset("ds_result").get_dataframe_from_csv().head()

### Step 2: Joining two datasets

In [ ]:
# getting dataframe of damage output
eq_bldg_dmg_df = eq_bldg_dmg.get_output_dataset("ds_result").get_dataframe_from_csv()

# getting geodataframe of building dataset 
bldg_gdf = local_buildings.get_dataframe_from_shapefile()

# merge/join two dataframe
# you can choose columns to be merged
bldg_dmg_gdf = bldg_gdf[['guid', 'struct_typ', 'geometry']].merge(eq_bldg_dmg_df, on='guid')
bldg_dmg_gdf.head()

## 2. Using Pandas

You can use Pandas and GeoPandas to dispaly tabular data and create charts. 
This secsion shows some examples.


In [ ]:
# Stat summary on a column
bldg_dmg_gdf["LS_0"].describe()

In [ ]:
# Show table sorted by DS_3 (decending) and struct_typ (ascending)

bldg_dmg_gdf.sort_values(['DS_3', 'struct_typ'], ascending=[0,1]).head()

In [ ]:
# Show table group by Structure type
grouped_bldg_dmg = bldg_dmg_gdf.groupby(by=['struct_typ'], as_index=True)\
.agg({'DS_0': 'mean', 'DS_1':'mean', 'DS_2': 'mean', 'DS_3': 'mean', 'guid': 'count'})
grouped_bldg_dmg.rename(columns={'guid': 'total_count'}, inplace=True)
grouped_bldg_dmg.head()

In [ ]:
# Plot Damage state by structure type
ax = grouped_bldg_dmg[["DS_0", "DS_1", "DS_2", "DS_3"]].plot.barh(stacked=True)
ax.set_title("Stacked Bar Chart of Damage State Grouped by Structure Type", fontsize=12)
ax.set_xlabel("complete damage value", fontsize=12)
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5)) #here is the magic

In [ ]:
# Create a histogram with DS_3

ax = bldg_dmg_gdf['DS_3'].hist(bins=20, figsize=[10,5])
ax.set_title("complete damage distribution", fontsize=12)
ax.set_xlabel("complete damage value", fontsize=12)
ax.set_ylabel("counts", fontsize=12)

## 3. Using pyIncore-viz

pyIncore-viz is a Python visualization package for pyIncore module. It provides users with convenient methods to create image and graphic outputs of various hazard IN-CORE analyses.

### 3.1 Plotting Fragility curves

In [ ]:
# visualize this fragility curve using pyincore-viz package
from pyincore_viz.plotutil import PlotUtil as plot

In [ ]:
# Plot fragility curves

fragility_set_id = "5b47b2d7337d4a36187c61ce"

# Get local fragility curve set
fragility_curve_set = FragilityCurveSet(fragility_services.get_dfr3_set(fragility_set_id))
fragility_curve_set.fragility_curves

plt = plot.get_fragility_plot(fragility_curve_set, title="Mid-Rise Steel Moment Frame")
plt.show()

In [ ]:
#plot fragility curves of tornado building by going through mapping

fragility_service = FragilityService(client)
mapping_id = "5d8942dfb9219c068ea795ea"
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))

# plot fragility for the first 3 archetypes using pyincore viz method
for mapping in mapping_set.mappings[:3]:
    fragility_id = mapping.entry['Non-Retrofit Fragility ID Code']
    fragility_set = FragilityCurveSet(fragility_service.get_dfr3_set(fragility_id))
    plt = plot.get_fragility_plot(fragility_set)
    plt.show()
    

In [ ]:
# Plot fragility curves (parametric curves)

# use utility method of pyicore-viz package to visulaize the fragility
fragility_set = FragilityCurveSet(FragilityService(client).get_dfr3_set("5f6ccf67de7b566bb71b202d"))
plt = plot.get_fragility_plot_3d_refactored(fragility_set, 
                                            title="Galveston empirical fragility model developed "
                                            "based on Hurricane Ike surveys",
                                            limit_state="LS_0")
plt.show()


### 3.2 Plot Geospatial Map

Plotting a geospatial map is generating static image of a map.

In [ ]:
from pyincore_viz.geoutil import GeoUtil as geoviz


In [ ]:
# Plot tornado path

tornado_path_id = "5d07cbeab9219c065b080930"
geoviz.plot_tornado(tornado_path_id, client, basemap=True)

In [ ]:
# Plot earthquake

eq_id = "5b902cb273c3371e1236b36b"
geoviz.plot_earthquake(eq_id, client)

In [ ]:
# Plot a map with a Dataset 
# with local dataset (local_buildings)

geoviz.plot_map(local_buildings, column='struct_typ',category='True')

In [ ]:
# Plot a map with GeoDataframe 

geoviz.plot_gdf_map(bldg_dmg_gdf, 'DS_3', basemap=False)

In [ ]:
# Plot a map with GeoDataframe 

geoviz.plot_gdf_map(bldg_dmg_gdf, 'DS_3', basemap=True)

In [ ]:
# Interactive map with Dataset
# addjust additional multiplier to increase the intensity of heatmap, e.g. multipler=1000 
map = geoviz.plot_heatmap_from_gdf(bldg_dmg_gdf, "DS_3", radius=7, blur=5, name="DS_3", multiplier=1000) 
map

## Hands-on Assignment

Please open "session4-assignment.ipynb"